In [11]:
# %pip install pandas
# %pip install pyarrow
# %pip install python-dotenv
# %pip install google-api-python-client

In [52]:
import pandas as pd
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
import numpy as np

In [69]:
def get_video_details(video_url):
    # Extract video ID from URL
    video_id = video_url.split('v=')[1]
    # Make API call to get video details
    video_response = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        id=video_id
    ).execute()

    if video_response['items']:
        video_details = video_response['items'][0]
        return {
            'Title': video_details['snippet']['title'],
            'Description': video_details['snippet']['description'],
            'Upload Date': video_details['snippet']['publishedAt'],
            'Duration': video_details['contentDetails']['duration'],
            'Views': video_details['statistics']['viewCount'],
            'Comments': video_details['statistics'].get('commentCount', '0'),  # Use '0' or None as default
            'Likes': video_details['statistics'].get('likeCount', '0'),
        }
    else:
        return None

In [58]:
def update_dataframe_row(row):
    # Check if any of the fields are already populated
    fields_to_check = ['Title', 'Description', 'Upload Date', 'Duration', 'Views']  # Assuming 'Views' is also a field you're updating
    if all(not pd.isnull(row.get(field, '')) and row.get(field, '') != '' for field in fields_to_check):
        # All fields are already populated, do not overwrite
        return row
    
    # If any of the details are missing, fetch from YouTube
    video_details = get_video_details(row['Link'])
    if video_details:
        for field in fields_to_check:
            # Update the field only if it's considered blank (NaN or empty string)
            if pd.isnull(row.get(field)) or row.get(field) == '':
                row[field] = video_details.get(field, None)
    return row

In [65]:
# set up the YouTube API
load_dotenv(verbose=True)
api_key = os.environ.get("YOUTUBE_API_KEY")
youtube = build('youtube', 'v3', developerKey=api_key)

# read the list of videos from the google sheet
# csv_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR5B9xWBbS5VaTEWiLQreeQyNmyAilb7Xt1doIAuDqF-ndpSWWRnrjnpxmFk24HWSjg83wW6GoEWn2Q/pub?gid=0&single=true&output=csv'
# newer version
csv_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR5B9xWBbS5VaTEWiLQreeQyNmyAilb7Xt1doIAuDqF-ndpSWWRnrjnpxmFk24HWSjg83wW6GoEWn2Q/pub?gid=1771724794&single=true&output=csv'
df = pd.read_csv(csv_url)
df = df.astype(str).replace('nan',None) # make it all objects and replace 'nan' with None

df.head(2)

,Type,Summary,Link,Link2,Link2 Type,Title,Description,Upload Date,Duration,Views,Comments,Likes,GitHub,Language,Library,File Size,Music
0,Math,Circle packing,https://www.bilibili.com/video/BV1rV4y1C7LB/,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Math,"Circle Packing (Python, CV2)",https://www.youtube.com/watch?v=37AqUjnsNKw,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [70]:
# get the details for each youtube video, don't overwrite data that's already there

for index, row in df.iterrows():
    link = row['Link']
    if link and 'youtube' in link:
        updated_row = update_dataframe_row(row)
        df.loc[index] = updated_row


In [71]:
df

,Type,Summary,Link,Link2,Link2 Type,Title,Description,Upload Date,Duration,Views,Comments,Likes,GitHub,Language,Library,File Size,Music
0,Math,Circle packing,https://www.bilibili.com/video/BV1rV4y1C7LB/,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Math,"Circle Packing (Python, CV2)",https://www.youtube.com/watch?v=37AqUjnsNKw,None,None,"Bad Apple, But It's a Fractal (Higher Quality)",I redid it with a higher resolution and smalle...,2022-09-22T07:00:05Z,PT3M40S,512,None,None,None,None,None,None,None
2,Math,Moving Bubbles,https://www.youtube.com/watch?v=aQPLFQsJl1w,None,None,"Bad Apple, but it's moving bubbles",,2023-08-19T13:33:58Z,PT3M40S,3236,None,None,None,None,None,None,None
3,Math,Fourier,https://www.youtube.com/watch?v=GGPjQGOCbTY,None,None,Bad Apple but rendered as a Fourier Series,excuse the jank\n\nclean version: https://yout...,2022-08-07T11:11:51Z,PT4M4S,247786,None,None,None,None,None,None,None
4,Math,Bezier,https://www.youtube.com/watch?v=MVrNn5TuMkY,https://www.youtube.com/watch?v=BQvBq3K50u8,Making of,Bad Apple!! played on Desmos but it's high qua...,The squeakquel. Taking a bezier curve approach...,2021-04-16T14:00:07Z,PT4M19S,2264747,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Real World (or animation or both),Live Action Shadows,https://www.youtube.com/watch?v=jEUEE6gjHsg,None,None,【Touhou】Bad Apple!! 【Shadow Art】【東方】Real Peopl...,someone in the comments suggested someone do t...,2013-04-28T01:15:21Z,PT4M5S,35420,None,None,None,None,None,None,None
215,Real World (or animation or both),Lego,https://www.youtube.com/watch?v=1zyITlqhdso,None,None,Bad Apple!! but Lego and rendered with way too...,Stepping on a Bad Apple doesn't hurt much... u...,2021-10-18T18:19:47Z,PT3M39S,11042,None,None,None,None,None,None,None
216,Real World (or animation or both),Dolls,https://www.youtube.com/watch?v=yXdB77UhBuE,None,None,スーパードルフィーが「Bad Apple!!」に合わせて踊る！,60cmサイズのドール型ロボット、ダンシングドールの亜里栖（アリス）です。\r\n【東方】B...,2010-05-01T12:33:12Z,PT3M40S,198914,None,None,None,None,None,None,None
217,Real World (or animation or both),Dolls 2,https://www.youtube.com/watch?v=-3YjtujQls0,None,None,ドール型ロボットが「Bad Apple!!」を踊る！,60cmサイズのドール型ロボット、ダンシングドールの亜里栖(ありす)＆輝夜(かぐや)です。\...,2013-05-26T08:44:28Z,PT3M40S,93860,None,None,None,None,None,None,None


In [72]:
df.to_csv('youtube_videos.csv', index=False, encoding='utf-8', sep='|')

In [20]:

# Example usage
video_url = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'  # Example YouTube video URL
details = get_video_details(video_url)
print(details)

{'Title': 'Rick Astley - Never Gonna Give You Up (Official Music Video)', 'Description': "The official video for “Never Gonna Give You Up” by Rick Astley. \n\nThe new album 'Are We There Yet?' is out now: Download here: https://RickAstley.lnk.to/AreWeThereYetFA/itunes\n\n“Never Gonna Give You Up” was a global smash on its release in July 1987, topping the charts in 25 countries including Rick’s native UK and the US Billboard Hot 100.  It also won the Brit Award for Best single in 1988. Stock Aitken and Waterman wrote and produced the track which was the lead-off single and lead track from Rick’s debut LP “Whenever You Need Somebody”.  The album was itself a UK number one and would go on to sell over 15 million copies worldwide.\n\nThe legendary video was directed by Simon West – who later went on to make Hollywood blockbusters such as Con Air, Lara Croft – Tomb Raider and The Expendables 2.  The video passed the 1bn YouTube views milestone on 28 July 2021.\n\nSubscribe to the official 